In [1]:
#Script to export required field of the documents

import csv
import os
import requests
import json
import time
import io
import datetime;
import pandas as pd
from tqdm import tqdm
import re

#--------------Inputs to script---------------------------

#1.Csv file with document ID's (document_ids.csv-column name=Document ID)

#2.Provide the requird field names(backend name in the fields array)
fields=['id','document_number__v','major_version_number__v','minor_version_number__v','status__v','type__v','subtype__v',
        'classification__v','title__v','governing_functional_area__c','change__c','product__v','process__c','impacted_facilities__v']
#3.Vault application url
DNS='https://amgensbx-amgencdocs.veevavault.com/api/v21.2/'

#4.Username and password
Username='snyayadh-adm@amgensbx.com'
Password='********'

#-----------------------------------------------------------



file = open(r'Document_info.csv', 'a')
obj_name=[]
df_info=pd.DataFrame()


def authenticate(): #To get Session Id
    
    endpoint = DNS+'auth' 
    parameters = {
        'username':Username,
        'password':Password
    }
    
    try: 
        response = requests.post(endpoint, data=parameters).json()
        if response['responseStatus'] != 'SUCCESS':
            raise Exception()
        else:
            return response['sessionId']
    except:
        error_message = response['errorType'] + ', Reason: ' + response['responseMessage']
        print (error_message)
        
        
def fetch_documents(session_id,endpoint,doc_id,ind): #To Fetch Details
    
    headers = {
        'Authorization': session_id
    }
    
    response = requests.get(endpoint, headers=headers)
    try :
        if response.json():
            response = response.json()
            if response['responseStatus'] != 'SUCCESS':
                file.write("Not available for export at "+str(datetime.datetime.now())+" IST")
                file.write('\n')
            else:
                doc=response['document']['document_number__v']
                val=[]
                flag=0
                for i in fields:
                    if i in response['document'].keys():
                        
                        if i in obj_name:
                            Url=DNS+'vobjects/'
                            temp=str(response['document'][i])
                            
                            if "[" in temp:
                                length=len(response['document'][i])
                                #print(length)
                                obj_value=[]
                                flag_o=0
                                for j in range(length):
                                        flag_o=0
                                        temp=response['document'][i]
                                        
                                        #new_temp=re.sub(r'[\[\'\]]','',temp)
                                        param=''+str(i)+'/'+ temp[j] +''
                                        endpoint_obj=Url+param
                                        #print(endpoint)
                                        response1 = requests.get(endpoint_obj, headers=headers)
                                    
                                        if response1.json():
                                            response1 = response1.json()
                                            if response1['responseStatus'] != 'SUCCESS':
                                                #df_info.loc[ind,i]="something went wrong with obj field1"
                                                if i in response['document'].keys():
                                                    df_info.loc[ind,i]=str(response['document'][i])+"wrong"
                                            else:
                                                obj_value.append(str(response1['data']['name__v']))
                                                flag_o=1
                                if(flag_o==1):    
                                    l=len(obj_value)
                                    ori_val=""
                                    for k in range(l):
                                        ori_val=ori_val+obj_value[k]+","
                                        
                                    df_info.loc[ind,i]=str(ori_val)   
                                
                            else:
                                temp=str(response['document'][i])
                                
                                param=''+str(i)+'/'+ temp +''
                                endpoint_obj=Url+param
                                #print(endpoint)
                                response1 = requests.get(endpoint_obj, headers=headers)
                                if response1.json():
                                    response1 = response1.json()
                                    if response1['responseStatus'] != 'SUCCESS':
                                        #df_info.loc[ind,i]="something went wrong with obj field"
                                        if i in response['document'].keys():
                                                    df_info.loc[ind,i]=str(response['document'][i])+"wrong"
                                    else:
                                        df_info.loc[ind,i]=str(response1['data']['name__v'])
                        else:
                            df_info.loc[ind,i]=str(response['document'][i])
                    else:
                           val.append(i)
                           flag=1
                        
                              
                        
                if(flag!=1):
                    file.write(f"exported successfully {doc} ")
                    file.write('\n')
                else:
                        
                        file.write(f"Document {doc} exported with following field/s are missing/not available for document-")
                        for i in val:
                            file.write(f" {i} ")
                            
                        file.write('\n')
                
                
                
    except:
        file.write(f"something went wrong")
        file.write('\n')
            
       
        
        
def execute(): #To Get document id's and required parameter to call method
    session_id = authenticate()
    print(session_id)
    df = pd.read_csv('document_ids.csv')
    
    try:
        
        for ind in tqdm(df.index): 
            doc_id = str(df['Document ID'][ind])
            URI = DNS+'objects/documents/'
            action = ''+str(doc_id)+''
            endpoint = URI + action
            
            get_object(session_id)
            fetch_documents(session_id,endpoint,doc_id,ind)
            
    finally :
        
        df_info.to_csv('Document_export.csv',index=False)
        #print(df_info)
        file.close()
        
        
        
def get_object(session_id):   #To get all object fields name

    headers = {
        'Authorization': session_id
    }
    endpoint=DNS+'metadata/vobjects'
    response = requests.get(endpoint, headers=headers)
    
    if response.json():
            
            response = response.json()
            obj_count=len(response['objects'])
            
            for n in range(obj_count):
                obj_name.append(response['objects'][n]['name'])
    
            
  
execute()

  0%|                                                                                           | 0/35 [00:00<?, ?it/s]

435727333D489F39C53DE2646A06A0C9F6BE4613F107D3CDE57E7AD94D0BC360891E8B66282FC150943443E19F97E7CB62710141E6A8D7E3500177A62A7BC98E


100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [00:49<00:00,  1.41s/it]
